In [ ]:
# !nvidia-smi

In [ ]:
!pip install nltk
!pip install --upgrade transformers
!pip install --upgrade sentence-transformers
!pip install --upgrade scikit-learn
!pip install --upgrade pandas
!pip install --upgrade matplotlib seaborn
!pip install --upgrade sentencepiece
!pip install --no-cache-dir transformers sentencepiece
!pip uninstall -y sentencepiece
!pip uninstall -y transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install plotly
!pip install pandas numpy scipy scikit-learn transformers sentence-transformers gower

Found existing installation: sentencepiece 0.2.0
Uninstalling sentencepiece-0.2.0:
  Successfully uninstalled sentencepiece-0.2.0
Found existing installation: transformers 4.45.1
Uninstalling transformers-4.45.1:
  Successfully uninstalled transformers-4.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 691.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from transformers import DebertaV2Tokenizer, DebertaV2Model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist, squareform
import gower
# import sentencepiece as smp

In [ ]:
data = pd.read_csv("cleaned_goEmotions.csv")
data.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
emotion_columns = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
                   'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
                   'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
                   'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
                   'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
data = data.dropna(subset=['text'])
data = data.reset_index(drop=True)
data[emotion_columns] = data[emotion_columns].astype(int)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=False)
model = AutoModel.from_pretrained("microsoft/deberta-v3-base")

/ssddata/home/mmv5513/colabenv/lib64/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(128100, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
    (dropout): StableDropout()
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0-11): 12 x DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=768, out_features=768, bias=True)
            (key_proj): Linear(in_features=768, out_features=768, bias=True)
            (value_proj): Linear(in_features=768, out_features=768, bias=True)
            (pos_dropout): StableDropout()
            (dropout): StableDropout()
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
        )
        (intermedia

In [ ]:
def get_embeddings(texts, tokenizer, model, device, batch_size=32):
    embeddings = []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=128)
            inputs = {key: value.to(device) for key, value in inputs.items()}
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.extend(batch_embeddings)
    return np.array(embeddings)


In [ ]:
texts = data['text'].tolist()
text_embeddings = get_embeddings(texts, tokenizer, model, device)
print(f"Embeddings shape: {text_embeddings.shape}")

Embeddings shape: (211225, 768)


In [ ]:
emotion_features = data[emotion_columns].values
combined_features = np.hstack((text_embeddings, emotion_features))

In [ ]:
scaler = StandardScaler()
text_embeddings_scaled = scaler.fit_transform(text_embeddings)

combined_features_scaled = np.hstack((text_embeddings_scaled, emotion_features))

In [ ]:
df_combined = pd.DataFrame(text_embeddings_scaled)
df_emotions = pd.DataFrame(emotion_features, columns=emotion_columns)

In [ ]:
df_for_gower = pd.concat([df_combined, df_emotions], axis=1)
gower_dist = gower.gower_matrix(df_for_gower)

In [ ]:
optimal_clusters = 81

agglo = AgglomerativeClustering(n_clusters=optimal_clusters, affinity='precomputed', linkage='average')
cluster_labels = agglo.fit_predict(gower_dist)


In [ ]:
data['cluster'] = cluster_labels


In [ ]:
data.to_csv('clustered_data_81.csv', index=False)

In [ ]:
def display_cluster_samples(df, cluster_num, num_samples=5):
    print(f"\nCluster {cluster_num} samples:")
    samples = df[df['cluster'] == cluster_num]['text'].sample(n=num_samples, random_state=42)
    for text in samples:
        print(f"- {text}")

for cluster_num in range(optimal_clusters):
    display_cluster_samples(data, cluster_num)


In [ ]:
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords

# Download stopwords if you haven't already
nltk.download('stopwords')

# Get the English stopwords list
stop_words = set(stopwords.words('english'))

def get_cluster_keywords(df, cluster_num, num_keywords=10):
    texts = df[df['cluster'] == cluster_num]['text']
    all_words = ' '.join(texts).lower()
    words = re.findall(r'\b\w+\b', all_words)

    # Filter out stopwords
    filtered_words = [word for word in words if word not in stop_words]

    # Get the most common words that aren't stopwords
    common_words = Counter(filtered_words).most_common(num_keywords)

    print(f"\nCluster {cluster_num} top keywords (excluding stopwords):")
    for word, count in common_words:
        print(f"{word}: {count}")

for cluster_num in range(optimal_clusters):
    get_cluster_keywords(data, cluster_num)


In [ ]:
# Reduce embeddings to 2D for visualization
pca_2d = PCA(n_components=2, random_state=42)
embeddings_2d = pca_2d.fit_transform(reduced_features)

# Plot clusters
plt.figure(figsize=(10, 7))
sns.scatterplot(
    x=embeddings_2d[:, 0],
    y=embeddings_2d[:, 1],
    hue=cluster_labels,
    palette=sns.color_palette('hsv', num_clusters)  # or any large palette
)
plt.title('Clusters of Samples')
plt.show()


In [ ]:
import numpy as np
unique_clusters, counts = np.unique(cluster_labels, return_counts=True)
print(dict(zip(unique_clusters, counts)))


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Reduce embeddings to 3D for visualization
pca_3d = PCA(n_components=3, random_state=42)
embeddings_3d = pca_3d.fit_transform(reduced_features)  # Apply PCA to reduce to 3 components

# Create a 3D scatter plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the data points in 3D space with colors based on the cluster labels
scatter = ax.scatter(
    embeddings_3d[:, 0],  # x axis
    embeddings_3d[:, 1],  # y axis
    embeddings_3d[:, 2],  # z axis
    c=cluster_labels,     # color by cluster label
    cmap='viridis',       # colormap (you can adjust this)
    s=10                  # size of points
)

# Add a color bar to indicate which colors represent which clusters
legend1 = ax.legend(*scatter.legend_elements(), title="Clusters", loc="upper right")
ax.add_artist(legend1)

# Set labels
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')

# Title of the plot
plt.title('3D Cluster Visualization')

# Show the plot
plt.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Create a DataFrame from the 3D PCA results
df_3d = pd.DataFrame({
    'PCA Component 1': embeddings_3d[:, 0],
    'PCA Component 2': embeddings_3d[:, 1],
    'PCA Component 3': embeddings_3d[:, 2],
    'Cluster': cluster_labels
})

# Create an interactive 3D scatter plot
fig = px.scatter_3d(df_3d,
                    x='PCA Component 1',
                    y='PCA Component 2',
                    z='PCA Component 3',
                    color='Cluster',
                    title='Interactive 3D Cluster Visualization')

# Show the interactive plot
fig.show()


In [ ]:
import plotly.express as px
import plotly.io as pio

# Set default renderer for Jupyter Notebooks
pio.renderers.default = 'notebook'

# Show the plot
fig.show()


In [ ]:
import plotly.express as px
from plotly.offline import plot

# Use plotly.offline.plot to render in Google Colab
plot(fig)
